In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.9.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.9.1 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but y

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torchtext.legacy import datasets
from torchtext.legacy import data


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL =data.LabelField(dtype = torch.float)

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

We can see how many examples are in each split by checking their length.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))
type(train_data)

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['I', 'sat', 'down', 'to', 'watch', 'this', 'film', 'with', 'much', 'trepidation', 'and', 'little', 'hope', '.', 'I', 'did', "n't", 'think', 'it', 'would', 'be', 'possible', 'for', 'this', 'film', 'to', 'live', 'up', 'to', 'its', 'subject', 'matter', '.', 'But', 'it', 'absolutely', 'did', ',', 'and', 'then', 'some', '.', 'First', ',', 'I', 'must', 'say', 'that', 'Jared', 'Harris', 'did', 'an', 'extraordinary', 'job', 'as', 'John', 'Lennon', '.', 'At', 'times', 'it', 'seemed', 'that', 'Harris', 'was', 'channeling', 'Lennon', '.', 'The', 'resemblance', 'was', 'often', 'uncanny', ',', 'and', 'he', 'clearly', 'studied', 'Lennon', "'s", 'mannerisms', 'and', 'vocal', 'inflections', '.', 'Aiden', 'Quinn', 'was', 'quite', 'good', 'as', 'McCartney', ',', 'also', 'bearing', 'a', 'striking', 'resemblance', 'to', 'Macca', ',', 'although', 'he', 'did', 'occasionally', 'trip', 'over', 'his', 'Scouse', 'accent.<br', '/><br'

torchtext.legacy.datasets.imdb.IMDB

In [ ]:
import pandas as pd
import json

In [ ]:
def load_data(path, type):

  df = []
  print(path)
  with open(path, encoding="utf-8") as f:
    for i in f:
      i = json.loads(i)
      # tweet = i['postText']
      # article_title = i['targetTitle']
      # article = ' '.join(i['targetParagraphs'])
      label = i['tags']

      # tweet = tweet[0]
      label = label[0]

      final_label = 0

      if type == "1vs1" and label != 'multi':
        final_label = (label == 'phrase' or label == 'phrases') 
      elif type == "012":
        if label == 'passage':
          final_label = 1
        elif label == 'multi':
          final_label = 2
      else:
        continue

      df += [
          {
              # 'text': tweet + " - " + article_title + article,
           'postText': i['postText'],
                    'targetParagraphs': i['targetParagraphs'],
                    'targetTitle': i['targetTitle'],
                    'targetDescription': i['targetDescription'],
                    'targetKeywords': i['targetKeywords'],
                    'spoiler': i['spoiler'],
              'label': final_label
          }
      ]

    return pd.DataFrame(df)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
def __len__(self):
        return len(self.labels)
def __getitem__(self, index):
        label = self.labels[index]
        text = self.text[index]
        sample = {"Text": text, "Class": label}
        return sample

In [ ]:
train_dataset_path = "/content/train.jsonl"
test_dataset_path = "/content/test.jsonl"
validation_dataset_path = "/content/validation.jsonl"

class_type = "1vs1"

train_data = load_data(train_dataset_path, class_type)
train_data

/content/train.jsonl


,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,label
0,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets date for full recovery of ozone hole],[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employees happy -- and it'...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect way to cook rice so that it's per...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook's video explosion completely sha...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[Cop Is Eating At A Chili's When Teen Hands Hi...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter account that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [ ]:
# import random

# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

valid_data = load_data(validation_dataset_path, class_type)
print(type(valid_data))

test_data = load_data(validation_dataset_path, class_type)


/content/validation.jsonl
<class 'pandas.core.frame.DataFrame'>
/content/validation.jsonl


In [ ]:
test_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,label
0,[Five Nights at Freddy’s Sequel Delayed for We...,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy's creator Scott Cawthon ...,None,[some of the plot elements are so disturbing t...,False
1,[Here’s how much you should be tipping your ha...,[Here’s how much you should be tipping your ha...,Here’s how much you should be tipping your hai...,Remembering how much you’re supposed to tip an...,,[20%],True
2,[A man swallowed a microSD card and you won't ...,[PetaPixel is one of my favorite blogs. The wr...,Man swallowed a microSD card and you won't bel...,PetaPixel is one of my favorite blogs. The wri...,None,[a man who swallowed a 64GB microSD card and t...,False
3,[This popular soda could cure your hangovers s...,[Most of us are familiar with the pounding hea...,"Sprite May Be The Best Hangover Cure, Chinese ...",Most of us are familiar with the pounding head...,"sprite hangover,sprite,hangover cure,hangover ...",[Sprite],True
4,[The anytime snack you won't feel guilty about...,[Whether you're between meals or craving a mid...,The Anytime Snack You Won't Feel Guilty About ...,Whether you're between meals or craving a midn...,"Healthy Snacks,healthy snack ideas,healthy sna...",[Smoky Paprika-Baked Garbanzo Beans],True
...,...,...,...,...,...,...,...
652,[Dog Dies One Hour After Hiking With His Owner...,[It was a routine walk for one Colorado woman ...,"Dog Dies One Hour After Hiking With His Owner,...",It was a routine walk for one Colorado woman a...,"Dog, Vet, Death, Water Hemlock, Colorado, Hike...",[the plant the dog was chewing on was deadly w...,False
653,[This is what happens when you leave a hotel c...,[Instead of encountering a mound of dirty towe...,This Is What Happens When You Leave A Hotel Cl...,Instead of encountering a mound of dirty towel...,"givebackfilms,give back films,video,random act...",[The video below shows the stunned cleaner ini...,False
654,[This Texas GOP elector announces that he won'...,[A Republican elector in Texas says he will no...,Texas GOP elector announces he won't vote for ...,The Electoral College vote for president on De...,"donald trump, texas, electoral college, faithl...",[Christopher Suprun],True
655,[WikiLeaks' Julian Assange Reported Dead],"[On 16 October 2016, WikiLeaks posted a series...","WikiLeaks’ Julian Assange Isn’t Dead, Just Off...",A series of mysterious tweets from WikiLeaks l...,"dead man's switch, julian assange, wikileaks",[Julian Assange’s internet link has been inten...,False


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
import spacy
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")
if not nlp.vocab:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def alter_nouns_with_synonyms(text):
    doc = nlp(text)
    result = []
    for token in doc:
        if token.pos_ == 'NOUN':
            synonyms = []
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms.append(lemma.name())
            if synonyms:
                changed = False
                for syn in synonyms:
                    if syn != token.text:
                        changed = True
                        result.append(syn)
                        break
                if not changed:
                    result.append(token.text)
            else:
                result.append(token.text)
        else:
            result.append(token.text)
    return ' '.join(result)

In [ ]:
def alter_data(change_set):
  df_result = []
  for i in range(len(change_set)):
    post_text = change_set.iloc[i]['postText'][0]
    altered_text = alter_nouns_with_synonyms(post_text)
    temp_res = {'postText': [altered_text],
                'targetParagraphs': change_set.iloc[i]['targetParagraphs'],
                'targetTitle': change_set.iloc[i]['targetTitle'],
                'targetDescription': change_set.iloc[i]['targetDescription'],
                'targetKeywords': change_set.iloc[i]['targetKeywords'],
                'spoiler': change_set.iloc[i]['spoiler'],
                'label': change_set.iloc[i]['label']}
    df_result.append(temp_res)
    altered_dataframe = pd.DataFrame(df_result, columns=['postText', 'targetParagraphs',
                                                     'targetTitle', 'targetDescription',
                                                     'targetKeywords', 'spoiler',
                                                     'label'])
  return altered_dataframe

In [ ]:
train_data = alter_data(train_data)
valid_data = alter_data(valid_data)


In [ ]:
train_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,label
0,"[Wes Welker Wanted Dinner With Tom Brady , But...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,[NASA sets day_of_the_month for full convalesc...,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,[This is what makes employee happy -- and it '...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,[The perfect manner to cook Rice so that it 's...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,[If You See A Purple Butterfly Sticker At The ...,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,[Has Facebook 's picture detonation completely...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,[bull Is Eating At A Chili 's When Teen Hands ...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,[You need to see this Twitter history that pre...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [ ]:
def lemmit(row):
  lemm_list = []
  for text in row['postText']:
    doc = nlp(text)
    for word in doc:
      if not word.is_punct and not word.is_stop:
        lemm_list.append(word.lemma_)
    break
  return ' '.join(lemm_list)

In [ ]:
train_data['postText'] = train_data.apply(lemmit, axis=1)
valid_data['postText'] = valid_data.apply(lemmit, axis=1)

In [ ]:
train_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,label
0,Wes Welker want Dinner Tom Brady Patriots QB B...,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,NASA set day_of_the_month convalescence ozone ...,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,make employee happy payroll_check,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,perfect manner cook Rice perfectly fluffy stic...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,happen new AirPods lose steal Apple,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,Purple Butterfly Sticker Hospital mean,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,Facebook picture detonation completely shake a...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,bull eat Chili Teen hand Folded Napkin walk aw...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,need Twitter history predict Beyoncé gestation,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [ ]:
def load_data(df):
    result = []

    for index, row in df.iterrows():
        tweet = row['postText']
        article_title = row['targetTitle']
        article = ' '.join(row['targetParagraphs'])
        label = row['label']
        result += [
            {
                'text': tweet + " - " + article_title + article,
                'label': label
            }
        ]
    return pd.DataFrame(result)

In [ ]:
train_data

,postText,targetParagraphs,targetTitle,targetDescription,targetKeywords,spoiler,label
0,Wes Welker want Dinner Tom Brady Patriots QB B...,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,",[how about that morning we go throw?],False
1,NASA set day_of_the_month convalescence ozone ...,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[2070],True
2,make employee happy payroll_check,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[intellectual stimulation],True
3,perfect manner cook Rice perfectly fluffy stic...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",[in a rice cooker],True
4,happen new AirPods lose steal Apple,[One of the biggest surprise announcements at ...,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,None,[Apple says that if AirPods are lost or stolen...,False
...,...,...,...,...,...,...,...
2636,Purple Butterfly Sticker Hospital mean,[If You See A Purple Butterfly Sticker At The ...,If You See A Purple Butterfly Sticker At The H...,Millie found out she was pregnant and she and ...,None,[one of her unborn twins had a condition calle...,False
2637,Facebook picture detonation completely shake a...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[it hasn’t necessarily taken the wind out of Y...,False
2638,bull eat Chili Teen hand Folded Napkin walk aw...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,"[It read, ""Thanks for keeping us safe.""]",False
2639,need Twitter history predict Beyoncé gestation,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",,,[@beyoncefan666],True


In [ ]:
train_data = load_data(train_data)
valid_data = load_data(valid_data)

In [ ]:
train_data

,text,label
0,Wes Welker want Dinner Tom Brady Patriots QB B...,False
1,NASA set day_of_the_month convalescence ozone ...,True
2,make employee happy payroll_check - Intellectu...,True
3,perfect manner cook Rice perfectly fluffy stic...,True
4,happen new AirPods lose steal Apple - Here's w...,False
...,...,...
2636,Purple Butterfly Sticker Hospital mean - If Yo...,False
2637,Facebook picture detonation completely shake a...,False
2638,bull eat Chili Teen hand Folded Napkin walk aw...,False
2639,need Twitter history predict Beyoncé gestation...,True


In [ ]:
valid_data

,text,label
0,Nights Freddy Sequel Delayed Weird Reason - Fi...,False
1,tip hairstylist - Here’s how much you should b...,True
2,adult_male swallow microSD identity_card will ...,False
3,popular sodium_carbonate cure hangover scienti...,True
4,anytime bite will feel guilty eat - The Anytim...,True
...,...,...
652,Dog die Hour hike Owner Veterinarian give Shoc...,False
653,happen leave hotel clean $ 500 gratuity - This...,False
654,Texas GOP voter announce will vote Trump - Tex...,True
655,WikiLeaks Julian Assange report dead - WikiLea...,False


In [ ]:
import pandas as pd
from torchtext.legacy.data import Field, Example, Dataset

def example_cmp(ex1, ex2):
    return len(ex1.text) < len(ex2.text)

def get_tt_from_df(df):
  examples = []
  for i, row in df.iterrows():
      text = row["text"]
      label = row["label"]
      example = Example.fromlist([text, label], fields=[("text", TEXT), ("label", LABEL)])
      examples.append(example)

  # Create a dataset from the list of examples
  dataset = Dataset(examples, fields=[("text", TEXT), ("label", LABEL)])
  return dataset

In [ ]:
train_data = get_tt_from_df(train_data)
valid_data = get_tt_from_df(valid_data)
# test_data = get_tt_from_df(test_data)
print(type(train_data))
print(type(valid_data))
# print(type(test_data))

<class 'torchtext.legacy.data.dataset.Dataset'>
<class 'torchtext.legacy.data.dataset.Dataset'>


In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "fasttext.simple.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/wiki.simple.vec: 293MB [00:18, 16.2MB/s]                           
100%|██████████| 111051/111051 [00:10<00:00, 10890.28it/s]


In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[(',', 68343), ('.', 57085), ('the', 56095), ('to', 34379), ('a', 29033), ('of', 28469), ('and', 26811), ('"', 24940), ('in', 20088), ('that', 15928), ('-', 15393), ('is', 12867), ('for', 10736), ('it', 10488), ('on', 9582), ("'s", 8523), ('The', 8352), ('I', 8300), ('you', 7959), ('with', 7935)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', ',', '.', 'the', 'to', 'a', 'of', 'and', '"']


In [ ]:
BATCH_SIZE = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

We are working with  cuda


## Bidirectional Multi-Layer LSTM




In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        #TO-DO
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, pad_idx)

        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(
            input_size = embedding_dim, 
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional
            )

        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=hidden_dim * 2 , out_features=output_dim)

        #4. Initialize Dropout
        self.dropout_layer = nn.Dropout(p=dropout)
        
    def forward(self, text, text_lengths):
        #text = [sent_len, batch_size]

        

        embeddings = self.embedding(text)
        dropout = self.dropout_layer(embeddings)
        output, (hidden, c0) = self.lstm(dropout)
        hidden = self.dropout_layer(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
            
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,220,857 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 300])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [ 0.2767, -0.3329, -0.2527,  ...,  0.1491,  0.3634,  0.2008],
        [ 0.0838, -0.1147,  0.1237,  ..., -0.5986, -0.3011,  0.1984],
        [-0.1617,  0.0979,  0.5174,  ...,  0.2056, -0.1916,  0.0966]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [ 0.2767, -0.3329, -0.2527,  ...,  0.1491,  0.3634,  0.2008],
        [ 0.0838, -0.1147,  0.1237,  ..., -0.5986, -0.3011,  0.1984],
        [-0.1617,  0.0979,  0.5174,  ...,  0.2056, -0.1916,  0.0966]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
from sklearn.metrics import confusion_matrix

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    final_predictions = None

    ttp, ttn, tfp, tfn = 0,0,0,0
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            final_predictions = predictions
            rounded_preds = torch.round(torch.sigmoid(final_predictions))

            final_predictions = rounded_preds.cpu().detach().numpy()
            true_labels = batch.label.cpu().detach().numpy()


            # calculate confusion matrix
            confusion = confusion_matrix(true_labels, final_predictions).ravel()

            # unpack TP, TN, FP, FN
            if len(confusion) == 4:
              tn, fp, fn, tp = confusion
              ttp += tp
              ttn += tn
              tfp += fp
              tfn += fn
          



            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()


    print(ttp, ttn, tfp, tfn)
    print((ttp + ttn) / (ttp+ttn+tfp+tfn))
    return epoch_loss / len(iterator), epoch_acc / len(iterator), final_predictions 

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
path = F"/content/gdrive/My Drive/bilstm_model.pt"
best_valid_loss = float('inf')

from sklearn.metrics import accuracy_score
final_predictions = None

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, final_predictions = evaluate(model, valid_iterator, criterion)

    model.train()
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

53 281 30 269
0.5276461295418642
Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 53.86%
	 Val. Loss: 0.687 |  Val. Acc: 54.24%
156 214 109 165
0.5745341614906833
Epoch: 02 | Epoch Time: 1m 8s
	Train Loss: 0.637 | Train Acc: 64.33%
	 Val. Loss: 0.693 |  Val. Acc: 58.48%
169 224 103 152
0.6064814814814815
Epoch: 03 | Epoch Time: 1m 5s
	Train Loss: 0.519 | Train Acc: 75.04%
	 Val. Loss: 0.679 |  Val. Acc: 61.36%
233 146 189 76
0.5885093167701864
Epoch: 04 | Epoch Time: 1m 10s
	Train Loss: 0.413 | Train Acc: 82.11%
	 Val. Loss: 0.743 |  Val. Acc: 59.85%
199 200 127 118
0.6195652173913043
Epoch: 05 | Epoch Time: 1m 6s
	Train Loss: 0.294 | Train Acc: 87.97%
	 Val. Loss: 0.965 |  Val. Acc: 62.88%


In [ ]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, pred = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%', pred)

169 224 103 152
0.6064814814814815
Test Loss: 0.679 | Test Acc: 61.36% [1.]
